# Empire 2018 Guest Witness Allocation Algorithm
Developed by Viran Weerasekera (February 2018).

In [1]:
import gurobipy as grb
import numpy as np
import pandas as pd

In [6]:
def initImport(rd):
    
    global has_repeated
    has_repeated = {}

    has_repeated["same_do-opp"] = {}
    has_repeated["opp_do-same"] = {}
    has_repeated["opp_dt-opp"] = {}
    has_repeated["opp_do-opp"] = {}


    global conflicts
    conflicts = {}
    
    global softconflicts
    softconflicts = {}

    global teams
    teams = []
    global sides
    sides = {}
    global dts #dts = Destination Teams
    dts = {}
    global dos #dos = Destination Oppositions
    dos = {}
    data_csv = pd.read_csv("data_afterr{}.csv".format(rd),index_col=0)
    conflicts_csv = pd.read_csv("conflicts.csv",index_col=0)

    # Create empty conflict dictionaries
    for ind,row in data_csv.iterrows():
        teams.append(ind)
        sides[ind] = {}
        sides[ind][1] = row[1]
        sides[ind][2] = row[4]
        sides[ind][3] = row[7]
    
        dts[ind] = {}
        dts[ind][1] = row[2]
        dts[ind][2] = row[5]
        dts[ind][3] = row[8]
    
        dos[ind] = {}
        dos[ind][1] = row[3]
        dos[ind][2] = row[6]
        dos[ind][3] = row[9]
    
    
    
        conflicts["{}_p".format(ind)] = []
        conflicts["{}_d".format(ind)] = []
        softconflicts["{}_p".format(ind)] = []
        softconflicts["{}_d".format(ind)] = []
    
    for ind,row in conflicts_csv.iterrows():
        team1 = ind
        team2 = row[0]
        
        if (team1 != "nan") and (team2 != "nan"):
            #1p, 2p
            if "{}_p".format(team2) not in conflicts["{}_p".format(team1)]:
                conflicts["{}_p".format(team1)].append("{}_p".format(team2))
            #1d, 2p
            if "{}_p".format(team2) not in conflicts["{}_d".format(team1)]:
                conflicts["{}_d".format(team1)].append("{}_p".format(team2))
            #1p, 2d
            if "{}_d".format(team2) not in conflicts["{}_p".format(team1)]:
                conflicts["{}_p".format(team1)].append("{}_d".format(team2))
            #1d, 2d
            if "{}_d".format(team2) not in conflicts["{}_d".format(team1)]:
                conflicts["{}_d".format(team1)].append("{}_d".format(team2))
            
            #2p, 1p
            if "{}_p".format(team1) not in conflicts["{}_p".format(team2)]:
                conflicts["{}_p".format(team2)].append("{}_p".format(team1))
            #2d, 1p
            if "{}_p".format(team1) not in conflicts["{}_d".format(team2)]:
                conflicts["{}_d".format(team2)].append("{}_p".format(team1))
            #2p, 1d
            if "{}_d".format(team1) not in conflicts["{}_p".format(team2)]:
                conflicts["{}_p".format(team2)].append("{}_d".format(team1))
            #2d, 1d
            if "{}_d".format(team1) not in conflicts["{}_d".format(team2)]:
                conflicts["{}_d".format(team2)].append("{}_d".format(team1))

def addConflicts(rd):
    for team in teams:
        
        if sides[team][1] == "p":
            #(1,1)
            if "{}_p".format(dts[team][1]) not in conflicts["{}_p".format(team)]:
                conflicts["{}_p".format(team)].append("{}_p".format(dts[team][1]))
            #(1,2)
            if "{}_d".format(dos[team][1]) not in conflicts["{}_p".format(team)]:
                conflicts["{}_p".format(team)].append("{}_p".format(dos[team][1]))
            #(1,3)
            if "{}_d".format(dts[team][1]) not in conflicts["{}_p".format(team)]:
                conflicts["{}_p".format(team)].append("{}_d".format(dts[team][1]))
            #(1,4)
            if "{}_p".format(dos[team][1]) not in conflicts["{}_p".format(team)]:
                conflicts["{}_p".format(team)].append("{}_p".format(dos[team][1]))
              
            ## Old Conflicts - Not used anymore
            #(2,1)
            #if "{}_p".format(dts[team][1]) not in conflicts["{}_d".format(team)]:
            #    conflicts["{}_d".format(team)].append("{}_p".format(dts[team][1]))
        
            
                    
            #(2,2)
            #if "{}_d".format(dos[team][1]) not in conflicts["{}_d".format(team)]:
            #    if "{}_d".format(dos[team][1]) not in softconflicts["{}_d".format(team)]:
            #        softconflicts["{}_d".format(team)].append("{}_d".format(dos[team][1]))
            #    else:
            #        conflicts["{}_d".format(team)].append("{}_d".format(dos[team][1]))
            
            
            #(2,3)
            #if "{}_d".format(dts[team][1]) not in conflicts["{}_d".format(team)]:
            #    if "{}_d".format(dts[team][1]) not in softconflicts["{}_d".format(team)]:
            #        softconflicts["{}_d".format(team)].append("{}_d".format(dts[team][1]))
            #    else:
            #        conflicts["{}_d".format(team)].append("{}_d".format(dts[team][1]))
                
            #(2,4)
            #if "{}_p".format(dos[team][1]) not in conflicts["{}_d".format(team)]:
            #    if "{}_p".format(dos[team][1]) not in softconflicts["{}_d".format(team)]:
            #        softconflicts["{}_d".format(team)].append("{}_p".format(dos[team][1]))
            #    else:
            #        conflicts["{}_d".format(team)].append("{}_p".format(dos[team][1]))
    
                
        elif sides[team][1] == "d":
            #(1,1)
            if "{}_d".format(dts[team][1]) not in conflicts["{}_d".format(team)]:
                conflicts["{}_d".format(team)].append("{}_p".format(dts[team][1]))
            #(1,2)
            if "{}_p".format(dos[team][1]) not in conflicts["{}_d".format(team)]:
                conflicts["{}_d".format(team)].append("{}_p".format(dos[team][1]))
            #(1,3)
            if "{}_p".format(dts[team][1]) not in conflicts["{}_d".format(team)]:
                conflicts["{}_d".format(team)].append("{}_d".format(dts[team][1]))
            #(1,4)
            if "{}_d".format(dos[team][1]) not in conflicts["{}_d".format(team)]:
                conflicts["{}_d".format(team)].append("{}_p".format(dos[team][1]))
            
            #(2,1)
            #if "{}_d".format(dts[team][1]) not in conflicts["{}_p".format(team)]:
            #    conflicts["{}_p".format(team)].append("{}_d".format(dts[team][1]))
            
            #(2,2)
            #if "{}_p".format(dos[team][1]) not in conflicts["{}_p".format(team)]:
            #    if "{}_p".format(dos[team][1]) not in softconflicts["{}_p".format(team)]:
            #        softconflicts["{}_p".format(team)].append("{}_p".format(dos[team][1]))
            #    else:
            #        conflicts["{}_p".format(team)].append("{}_p".format(dos[team][1]))
            
            
            #(2,3)
            #if "{}_p".format(dts[team][1]) not in conflicts["{}_p".format(team)]:
            #    if "{}_p".format(dts[team][1]) not in softconflicts["{}_p".format(team)]:
            #        softconflicts["{}_p".format(team)].append("{}_p".format(dts[team][1]))
            #    else:
            #        conflicts["{}_p".format(team)].append("{}_p".format(dts[team][1]))
                
            #(2,4)
            #if "{}_d".format(dos[team][1]) not in conflicts["{}_p".format(team)]:
            #    if "{}_d".format(dos[team][1]) not in softconflicts["{}_p".format(team)]:
            #        softconflicts["{}_p".format(team)].append("{}_d".format(dos[team][1]))
            #    else:
            #        conflicts["{}_p".format(team)].append("{}_d".format(dos[team][1]))
                
            
            
        if rd >= 3:
            if sides[team][2] == "p":
                #(1,1)
                if "{}_p".format(dts[team][2]) not in conflicts["{}_p".format(team)]:
                    conflicts["{}_p".format(team)].append("{}_p".format(dts[team][2]))
                #(1,2)
                if "{}_d".format(dos[team][2]) not in conflicts["{}_p".format(team)]:
                    conflicts["{}_p".format(team)].append("{}_p".format(dos[team][2]))
                #(1,3)
                if "{}_d".format(dts[team][2]) not in conflicts["{}_p".format(team)]:
                    conflicts["{}_p".format(team)].append("{}_d".format(dts[team][2]))
                #(1,4)
                if "{}_p".format(dos[team][2]) not in conflicts["{}_p".format(team)]:
                    conflicts["{}_p".format(team)].append("{}_p".format(dos[team][2]))
                
                #(2,1)
                #if "{}_p".format(dts[team][2]) not in conflicts["{}_d".format(team)]:
                #    conflicts["{}_d".format(team)].append("{}_p".format(dts[team][2]))
                    
                #(2,2)
                #if "{}_d".format(dos[team][2]) not in conflicts["{}_d".format(team)]:
                #    if "{}_d".format(dos[team][2]) not in softconflicts["{}_d".format(team)]:
                #        softconflicts["{}_d".format(team)].append("{}_d".format(dos[team][2]))
                #    else:
                #        conflicts["{}_d".format(team)].append("{}_d".format(dos[team][2]))
            
                #(2,3)
                #if "{}_d".format(dts[team][2]) not in conflicts["{}_d".format(team)]:
                #    if "{}_d".format(dts[team][2]) not in softconflicts["{}_d".format(team)]:
                #        softconflicts["{}_d".format(team)].append("{}_d".format(dts[team][2]))
                #    else:
                #        conflicts["{}_d".format(team)].append("{}_d".format(dts[team][2]))
                
                #(2,4)
                #if "{}_p".format(dos[team][2]) not in conflicts["{}_d".format(team)]:
                #    if "{}_p".format(dos[team][2]) not in softconflicts["{}_d".format(team)]:
                #        softconflicts["{}_d".format(team)].append("{}_p".format(dos[team][2]))
                #    else:
                #        conflicts["{}_d".format(team)].append("{}_p".format(dos[team][2]))
                    
            elif sides[team][2] == "d":
                #(1,1)
                if "{}_d".format(dts[team][2]) not in conflicts["{}_d".format(team)]:
                    conflicts["{}_d".format(team)].append("{}_p".format(dts[team][2]))
                #(1,2)
                if "{}_p".format(dos[team][2]) not in conflicts["{}_d".format(team)]:
                    conflicts["{}_d".format(team)].append("{}_p".format(dos[team][2]))
                #(1,3)
                if "{}_p".format(dts[team][2]) not in conflicts["{}_d".format(team)]:
                    conflicts["{}_d".format(team)].append("{}_d".format(dts[team][2]))
                #(1,4)
                if "{}_d".format(dos[team][2]) not in conflicts["{}_d".format(team)]:
                    conflicts["{}_d".format(team)].append("{}_p".format(dos[team][2]))
                
                #(2,1)
                #if "{}_d".format(dts[team][2]) not in conflicts["{}_p".format(team)]:
                #    conflicts["{}_p".format(team)].append("{}_d".format(dts[team][2]))
                    
                #(2,2)
                #if "{}_p".format(dos[team][2]) not in conflicts["{}_p".format(team)]:
                #    if "{}_p".format(dos[team][2]) not in softconflicts["{}_p".format(team)]:
                #        softconflicts["{}_p".format(team)].append("{}_p".format(dos[team][2]))
                #    else:
                #        conflicts["{}_p".format(team)].append("{}_p".format(dos[team][2]))
            
            
                #(2,3)
                #if "{}_p".format(dts[team][2]) not in conflicts["{}_p".format(team)]:
                #    if "{}_p".format(dts[team][2]) not in softconflicts["{}_p".format(team)]:
                #        softconflicts["{}_p".format(team)].append("{}_p".format(dts[team][2]))
                #    else:
                #        conflicts["{}_p".format(team)].append("{}_p".format(dts[team][2]))

                #(2,4)
                #if "{}_d".format(dos[team][2]) not in conflicts["{}_p".format(team)]:
                #    if "{}_d".format(dos[team][2]) not in softconflicts["{}_p".format(team)]:
                #        softconflicts["{}_p".format(team)].append("{}_d".format(dos[team][2]))
                #    else:
                #        conflicts["{}_p".format(team)].append("{}_d".format(dos[team][2]))
                        
        if rd == 4:
            if sides[team][3] == "p":
                #(1,1)
                if "{}_p".format(dts[team][3]) not in conflicts["{}_p".format(team)]:
                    conflicts["{}_p".format(team)].append("{}_p".format(dts[team][3]))
                #(1,2)
                if "{}_d".format(dos[team][3]) not in conflicts["{}_p".format(team)]:
                    conflicts["{}_p".format(team)].append("{}_p".format(dos[team][3]))
                #(1,3)
                if "{}_d".format(dts[team][3]) not in conflicts["{}_p".format(team)]:
                    conflicts["{}_p".format(team)].append("{}_d".format(dts[team][3]))
                #(1,4)
                if "{}_p".format(dos[team][3]) not in conflicts["{}_p".format(team)]:
                    conflicts["{}_p".format(team)].append("{}_p".format(dos[team][3]))
                 
                #(2,1)
                #if "{}_p".format(dts[team][3]) not in conflicts["{}_d".format(team)]:
                #    conflicts["{}_d".format(team)].append("{}_p".format(dts[team][3]))
                
                #(2,2)
                #if "{}_d".format(dos[team][3]) not in conflicts["{}_d".format(team)]:
                #    if "{}_d".format(dos[team][3]) not in softconflicts["{}_d".format(team)]:
                #        softconflicts["{}_d".format(team)].append("{}_d".format(dos[team][3]))
                #    else:
                #        conflicts["{}_d".format(team)].append("{}_d".format(dos[team][3]))
            
                #(2,3)
                #if "{}_d".format(dts[team][3]) not in conflicts["{}_d".format(team)]:
                #    if "{}_d".format(dts[team][3]) not in softconflicts["{}_d".format(team)]:
                #        softconflicts["{}_d".format(team)].append("{}_d".format(dts[team][3]))
                #    else:
                #        conflicts["{}_d".format(team)].append("{}_d".format(dts[team][3]))
                
                #(2,4)
                #if "{}_p".format(dos[team][3]) not in conflicts["{}_d".format(team)]:
                #    if "{}_p".format(dos[team][3]) not in softconflicts["{}_d".format(team)]:
                #        softconflicts["{}_d".format(team)].append("{}_p".format(dos[team][3]))
                #    else:
                #        conflicts["{}_d".format(team)].append("{}_p".format(dos[team][3]))
                        
            elif sides[team][3] == "d":
                #(1,1)
                if "{}_d".format(dts[team][3]) not in conflicts["{}_d".format(team)]:
                    conflicts["{}_d".format(team)].append("{}_p".format(dts[team][3]))
                #(1,2)
                if "{}_p".format(dos[team][3]) not in conflicts["{}_d".format(team)]:
                    conflicts["{}_d".format(team)].append("{}_p".format(dos[team][3]))
                #(1,3)
                if "{}_p".format(dts[team][3]) not in conflicts["{}_d".format(team)]:
                    conflicts["{}_d".format(team)].append("{}_d".format(dts[team][3]))
                #(1,4)
                if "{}_d".format(dos[team][3]) not in conflicts["{}_d".format(team)]:
                    conflicts["{}_d".format(team)].append("{}_p".format(dos[team][3]))
                    
                #(2,1)
                #if "{}_d".format(dts[team][3]) not in conflicts["{}_p".format(team)]:
                #    conflicts["{}_p".format(team)].append("{}_d".format(dts[team][3]))
                
                #(2,2)
                #if "{}_p".format(dos[team][3]) not in conflicts["{}_p".format(team)]:
                #    if "{}_p".format(dos[team][3]) not in softconflicts["{}_p".format(team)]:
                #        softconflicts["{}_p".format(team)].append("{}_p".format(dos[team][3]))
                #    else:
                #        conflicts["{}_p".format(team)].append("{}_p".format(dos[team][3]))
            
                #(2,3)
                #if "{}_p".format(dts[team][3]) not in conflicts["{}_p".format(team)]:
                #    if "{}_p".format(dts[team][3]) not in softconflicts["{}_p".format(team)]:
                #        softconflicts["{}_p".format(team)].append("{}_p".format(dts[team][3]))
                #    else:
                #        conflicts["{}_p".format(team)].append("{}_p".format(dts[team][3]))

                #(2,4)
                #if "{}_d".format(dos[team][3]) not in conflicts["{}_p".format(team)]:
                #    if "{}_d".format(dos[team][3]) not in softconflicts["{}_p".format(team)]:
                #        softconflicts["{}_p".format(team)].append("{}_d".format(dos[team][3]))
                #    else:
                #        conflicts["{}_p".format(team)].append("{}_d".format(dos[team][3]))
                        


def generateCosts(rd,scCost):
    global masterpairings
    global pairings
    global roomranks
    global costs
    global softconflictcosts
    global powercosts
    
    masterpairings = {}
    pairings = {}
    pairings[1] = {}
    pairings[2] = {}
    roomranks = {}
    costs = {}
    softconflictcosts = {}
    powercosts = {}
    
    
    div1pairings_csv = pd.read_csv("pairingsr{}_div1.csv".format(rd),index_col=0)
    div2pairings_csv = pd.read_csv("pairingsr{}_div2.csv".format(rd),index_col=0)
    
    for ind,row in div1pairings_csv.iterrows():
        pairings[1][row[0]] = [row[1],row[2]]
        masterpairings[row[0]] = [row[1],row[2]]
        roomranks[row[0]] = ind
    
    for ind,row in div2pairings_csv.iterrows():
        pairings[2][row[0]] = [row[1],row[2]]
        masterpairings[row[0]] = [row[1],row[2]]
        roomranks[row[0]] = ind
    
    print(masterpairings)
    for div1room,div1pairing in pairings[1].iteritems():
        for div2room,div2pairing in pairings[2].iteritems():
            costs[div1room,div2room] = 0
            costs[div2room,div1room] = 0
            
            softconflictcosts[div1room,div2room] = 0
            softconflictcosts[div2room,div1room] = 0
            
            powercosts[div1room,div2room] = 0
            powercosts[div2room,div1room] = 0

    for assignment,cost in costs.iteritems():
        originPairing = masterpairings[assignment[0]]
        destinationPairing = masterpairings[assignment[1]]
        
        costs[assignment] += 100*np.absolute(roomranks[assignment[0]] - roomranks[assignment[1]])
        powercosts[assignment] += 100*np.absolute(roomranks[assignment[0]] - roomranks[assignment[1]])

        ## SOFT CONFLICTS
         # Checks if destination p is conflicted off origin p
        #if "{}_p".format(destinationPairing[0]) in softconflicts["{}_p".format(originPairing[0])]:
        #    #print("{} -> {} - {}_p has soft conflict with {}_p".format(assignment[0],assignment[1],destinationPairing[0],originPairing[0]))
        #    costs[assignment] += scCost
        #    softconflictcosts[assignment] += scCost
        # Checks if destination d is conflicted off origin p
        #if "{}_d".format(destinationPairing[1]) in softconflicts["{}_p".format(originPairing[0])]:
        #    #print("{} -> {} - {}_p has soft conflict with {}_p".format(assignment[0],assignment[1],destinationPairing[0],originPairing[0]))
        #    costs[assignment] += scCost
        #    softconflictcosts[assignment] += scCost
        # Checks if destination p is conflicted off origin d
        #if "{}_p".format(destinationPairing[0]) in softconflicts["{}_p".format(originPairing[1])]:
        #    #print("{} -> {} - {}_p has soft conflict with {}_p".format(assignment[0],assignment[1],destinationPairing[0],originPairing[0]))
        #    costs[assignment] += scCost
        #    softconflictcosts[assignment] += scCost
        # Checks if destination d is conflicted off origin d
        #if "{}_d".format(destinationPairing[1]) in softconflicts["{}_p".format(originPairing[1])]:
        #    #print("{} -> {} - {}_p has soft conflict with {}_p".format(assignment[0],assignment[1],destinationPairing[0],originPairing[0]))
        #    costs[assignment] += scCost
        #    softconflictcosts[assignment] += scCost
            
        ## Checks if origin p is conflicted off destination p
        #if "{}_p".format(originPairing[0]) in softconflicts["{}_p".format(destinationPairing[0])]:
        #    #print("{} -> {} - {}_p has soft conflict with {}_p".format(assignment[1],assignment[0],originPairing[0],destinationPairing[0]))
        #    costs[assignment] += scCost
        #    softconflictcosts[assignment] += scCost
        ## Checks if origin d is conflicted off destination p
        #if "{}_d".format(originPairing[1]) in softconflicts["{}_p".format(destinationPairing[0])]:
        #    #print("{} -> {} - {}_p has soft conflict with {}_p".format(assignment[1],assignment[0],originPairing[0],destinationPairing[0]))
        #    costs[assignment] += scCost
        #    softconflictcosts[assignment] += scCost
        # Checks if origin p is conflicted off destination d
        #if "{}_p".format(originPairing[0]) in softconflicts["{}_p".format(destinationPairing[1])]:
        #    #print("{} -> {} - {}_p has soft conflict with {}_p".format(assignment[1],assignment[0],originPairing[0],destinationPairing[0]))
        #    costs[assignment] += scCost
        #    softconflictcosts[assignment] += scCost
        # Checks if origin d is conflicted off destination d
        #if "{}_d".format(originPairing[1]) in softconflicts["{}_p".format(destinationPairing[1])]:
        #    #print("{} -> {} - {}_p has soft conflict with {}_p".format(assignment[1],assignment[0],originPairing[0],destinationPairing[0]))
        #    costs[assignment] += scCost
        #    softconflictcosts[assignment] += scCost
            
            
        ## HARD CONFLICTS
        # Checks if destination p is conflicted off origin p
        if "{}_p".format(destinationPairing[0]) in conflicts["{}_p".format(originPairing[0])]:
            #print("{} -> {} - {}_p has conflict with {}_p".format(assignment[0],assignment[1],destinationPairing[0],originPairing[0]))
            costs[assignment] = grb.GRB.INFINITY
        # Checks if destination d is conflicted off origin p
        if "{}_d".format(destinationPairing[1]) in conflicts["{}_p".format(originPairing[0])]:
            #print("{} -> {} - {}_p has conflict with {}_p".format(assignment[0],assignment[1],destinationPairing[0],originPairing[0]))
            costs[assignment] = grb.GRB.INFINITY
        # Checks if destination p is conflicted off origin d
        if "{}_p".format(destinationPairing[0]) in conflicts["{}_p".format(originPairing[1])]:
            #print("{} -> {} - {}_p has conflict with {}_p".format(assignment[0],assignment[1],destinationPairing[0],originPairing[0]))
            costs[assignment] = grb.GRB.INFINITY
        # Checks if destination d is conflicted off origin d
        if "{}_d".format(destinationPairing[1]) in conflicts["{}_p".format(originPairing[1])]:
            #print("{} -> {} - {}_p has conflict with {}_p".format(assignment[0],assignment[1],destinationPairing[0],originPairing[0]))
            costs[assignment] = grb.GRB.INFINITY
            
        # Checks if origin p is conflicted off destination p
        if "{}_p".format(originPairing[0]) in conflicts["{}_p".format(destinationPairing[0])]:
            #print("{} -> {} - {}_p has conflict with {}_p".format(assignment[1],assignment[0],originPairing[0],destinationPairing[0]))
            costs[assignment] = grb.GRB.INFINITY
        # Checks if origin d is conflicted off destination p
        if "{}_d".format(originPairing[1]) in conflicts["{}_p".format(destinationPairing[0])]:
            #print("{} -> {} - {}_p has conflict with {}_p".format(assignment[1],assignment[0],originPairing[0],destinationPairing[0]))
            costs[assignment] = grb.GRB.INFINITY
        # Checks if origin p is conflicted off destination d
        if "{}_p".format(originPairing[0]) in conflicts["{}_p".format(destinationPairing[1])]:
            #print("{} -> {} - {}_p has conflict with {}_p".format(assignment[1],assignment[0],originPairing[0],destinationPairing[0]))
            costs[assignment] = grb.GRB.INFINITY
        # Checks if origin d is conflicted off destination d
        if "{}_d".format(originPairing[1]) in conflicts["{}_p".format(destinationPairing[1])]:
            #print("{} -> {} - {}_p has conflict with {}_p".format(assignment[1],assignment[0],originPairing[0],destinationPairing[0]))
            costs[assignment] = grb.GRB.INFINITY


def allocate(rd,scCost):
    m = grb.Model("Guest Witness Allocation - Round {}".format(rd))
    x = {}
    for key,value in costs.iteritems():
        x[key] = m.addVar(vtype = grb.GRB.BINARY, obj = value, name = "x_{}".format(value))

    m.modelSense = grb.GRB.MINIMIZE
    
    div1Rooms = []
    div2Rooms = []
    
    div1Rooms = pairings[1].keys()
    div2Rooms = pairings[2].keys()

    for room in masterpairings.keys():
        if room in div1Rooms:
            m.addConstr(sum(x[room,room2] for room2 in div2Rooms) == 1)
            m.addConstr(sum(x[room2,room] for room2 in div2Rooms) == 1)
        elif room in div2Rooms:
            m.addConstr(sum(x[room,room2] for room2 in div1Rooms) == 1)
            m.addConstr(sum(x[room2,room] for room2 in div1Rooms) == 1)
    
    
    m.optimize()
    
    print("------------------------------------".format(rd))
    print("Guest Witness Allocations - Round {}".format(rd))
    
    roomsToPrint = []
    roomsToPrint = masterpairings.keys()
    roomsToPrint.sort()
    
    allocatedOrigins = []
    allocatedDestinations = []
    
    d1outputRooms = [["From","To"]]
    d2outputRooms = [["From","To"]]
    
    for room in roomsToPrint:
        if room in div1Rooms:
            for room2 in div2Rooms:
                if x[room,room2].x != 0:
                    print("{} sends GWs to {} ({}) ({} Room Power Diff, {} SCs)".format(room,room2,costs[room,room2],powercosts[room,room2],softconflictcosts[room,room2]/scCost))
                    d1outputRooms.append([room,room2])
                    if room in allocatedOrigins:
                        print("origin")
                    else:
                        allocatedOrigins.append(room)
                        
                    if room2 in allocatedDestinations:
                        print("destination")
                    else:
                        allocatedDestinations.append(room2)     
        elif room in div2Rooms:
            for room2 in div1Rooms:
                if x[room,room2].x != 0:
                    print("{} sends GWs to {} ({}) ({} Room Power Diff, {} SCs)".format(room,room2,costs[room,room2],powercosts[room,room2],softconflictcosts[room,room2]/scCost))
                    d2outputRooms.append([room,room2])
                    if room in allocatedOrigins:
                        print("origin")
                    else:
                        allocatedOrigins.append(room)
                        
                    if room2 in allocatedDestinations:
                        print("destination")
                    else:
                        allocatedDestinations.append(room2)
    pdOutputArray1 = pd.DataFrame(d1outputRooms)
    pdOutputArray2 = pd.DataFrame(d2outputRooms)
    pdOutputArray1.to_csv("assignments_r{}_div1.csv".format(rd))
    pdOutputArray2.to_csv("assignments_r{}_div2.csv".format(rd))

In [7]:
nextRound = 4
lastRound = nextRound - 1
softCost = -1
# Deprioritise/neutralise softCost in Round 2, PRIORITISE in Round 3

In [10]:
initImport(lastRound)
print(conflicts)
print("-----")
addConflicts(nextRound)
#for key,value in conflicts.iteritems():
#    print("{}: {}".format(key,value))
#print("--------")
generateCosts(nextRound,softCost)
allocate(nextRound,softCost)

{'Samios_p': [], 'Faiella_p': [], 'Coulibaly_p': [], 'Denis_d': [], 'Morton_p': [], 'Cepeda_p': [], 'Faiella_d': [], 'Denis_p': [], 'Samios_d': [], 'Smith_d': [], 'Cepeda_d': [], 'Weerasekera_p': ['Burdette_p', 'Burdette_d'], 'Matarrese_d': [], 'Burdette_d': ['Weerasekera_p', 'Weerasekera_d'], 'Rosier_p': [], 'Gossett_d': [], 'Beeforth_p': [], 'Dawson_d': [], 'Burdette_p': ['Weerasekera_p', 'Weerasekera_d'], 'Rosier_d': [], 'Weerasekera_d': ['Burdette_p', 'Burdette_d'], 'Matarrese_p': [], 'Dawson_p': [], 'Beeforth_d': [], 'Abry_p': [], 'Divins_p': [], 'Pym_p': [], 'Franklin_d': [], 'Abry_d': [], 'Hahn_p': [], 'Stebbins_d': [], 'Franklin_p': [], 'Coulibaly_d': [], 'Benjamin_p': [], 'Morton_d': [], 'Pym_d': [], 'Benjamin_d': [], 'Hahn_d': [], 'Stebbins_p': [], 'Gossett_p': [], 'Fast_d': [], 'Bell-Colpack_d': [], 'Smith_p': [], 'Gramajo_d': [], 'Self_p': [], 'Shapiro_d': [], 'Felder_p': [], 'Kelly_p': [], 'Gramajo_p': [], 'Fast_p': [], 'Bell-Colpack_p': [], 'Divins_d': [], 'Kelly_d': [], 

In [5]:
print(conflicts)

{'Samios_p': ['Shapiro_p', 'Faiella_p', 'Shapiro_d'], 'Faiella_p': ['Cepeda_p', 'Weerasekera_p', 'Cepeda_d', 'Franklin_p', 'Benjamin_p', 'Franklin_d'], 'Coulibaly_p': ['Benjamin_p', 'Self_p', 'Benjamin_d'], 'Denis_d': ['Cepeda_d', 'Hahn_d', 'Cepeda_p', 'Self_d', 'Matarrese_d', 'Self_p'], 'Morton_p': ['Gossett_p', 'Matarrese_p', 'Gossett_d', 'Weerasekera_p', 'Hahn_p', 'Weerasekera_d'], 'Cepeda_p': ['Faiella_p', 'Gramajo_p', 'Faiella_d', 'Dawson_p', 'Felder_p', 'Dawson_d'], 'Faiella_d': ['Gossett_d', 'Samios_d', 'Gossett_p'], 'Denis_p': ['Fast_p', 'Pym_p', 'Fast_d'], 'Samios_d': ['Burdette_d', 'Dawson_d', 'Burdette_p', 'Felder_d', 'Felder_p'], 'Smith_d': ['Bell-Colpack_d', 'Kelly_d', 'Bell-Colpack_p', 'Pym_d', 'Kelly_p'], 'Cepeda_d': ['Denis_d', 'Burdette_d', 'Denis_p'], 'Weerasekera_p': ['Burdette_p', 'Burdette_d', 'Stebbins_p', 'Coulibaly_p', 'Stebbins_d', 'Morton_p', 'Beeforth_p', 'Morton_d'], 'Matarrese_d': ['Abry_d', 'Morton_d', 'Abry_p'], 'Burdette_d': ['Weerasekera_p', 'Weeraseker